flipped_img = cv2.flip(img, flipCode)
- flipCode là 0, ảnh sẽ được lật theo chiều ngang.
- flipCode là 1, ảnh sẽ được lật theo chiều dọc.
- flipCode là -1, ảnh sẽ được lật cả hai chiều, tức là lật ngang và dọc cùng một lúc.

objects = ace_detector.detectMultiScale(image, scaleFactor, minNeighbors, minSize, maxSize): phát hiện đối tượng trong ảnh sử dụng Cascade Classifier
- minNeighbors: Là một ngưỡng để loại bỏ các kết quả không đáng tin cậy. Các khu vực được coi là đối tượng chỉ được giữ lại nếu chúng có một số lượng lân cận (neighbors) lớn hơn giá trị này.

In [10]:
import cv2

cam = cv2.VideoCapture('http://192.168.0.104:8080/video')
cam.set(3, 640)
cam.set(4, 480)

# file chỉ phát hiện khuôn mặt
face_detertor = cv2.CascadeClassifier('data\\haarcascade_frontalface_default.xml')
face_id = input('\n Nhap ID Khuon mat <return> ==> ')

print("\n [INFO] Khoi tao camera ...")
count = 0
while 1:
    _, img = cam.read()
    img = cv2.flip(img, -1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_detertor.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y + h), (255, 0, 0), 2)
        count += 1

        cv2.imwrite("data/datasetFace/User." + str(face_id) +"." + str(count) + ".jpg", gray[y:y+h, x: x+w])
        # cv2.imshow('image', img)
        print(f"saved picture {count}")
    if (cv2.waitKey(2) & 0xFF == 27) or count >= 30:
        break
print("\n [INFO] Thoat")
cam.release()
cv2.destroyAllWindows()


 [INFO] Khoi tao camera ...
saved picture 1
saved picture 2
saved picture 3
saved picture 4
saved picture 5
saved picture 6
saved picture 7
saved picture 8
saved picture 9
saved picture 10
saved picture 11
saved picture 12
saved picture 13
saved picture 14
saved picture 15
saved picture 16
saved picture 17
saved picture 18
saved picture 19
saved picture 20
saved picture 21
saved picture 22
saved picture 23
saved picture 24
saved picture 25
saved picture 26
saved picture 27
saved picture 28
saved picture 29
saved picture 30

 [INFO] Thoat


In [8]:
import cv2
import numpy as np
from PIL import Image
import os

path= "data/datasetFace"

recognizer = cv2.face.LBPHFaceRecognizer_create()
# recognizer = cv2.createLBPHFaceRecognizer()

detector = cv2.CascadeClassifier('data\\haarcascade_frontalface_default.xml')

def getImagesAndLabels(path):

    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
    faceSamples = []
    ids = []

    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # convert to grayscale
        img_numpy = np.array(PIL_img, 'uint8')
        id = int(os.path.split(imagePath)[-1].split('.')[1])
        faces = detector.detectMultiScale(img_numpy)

        for (x, y, w, h) in faces:
            faceSamples.append(img_numpy[y:y+h, x:x+w])
            ids.append(id)
    
    return faceSamples, ids

print("\n [INFO] Dang trainning du lieu ...")

faces, ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

recognizer.write("data/trainer/trainer.yml")

print("\n [INFO] {0} khuon mat duoc train. Thoat". format(len(np.unique(ids))))



 [INFO] Dang trainning du lieu ...

 [INFO] 1 khuon mat duoc train. Thoat


In [4]:
import cv2
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('data/trainer/trainer.yml')

cascalePath = "data/haarcascade_frontalface_default.xml"
faceCascale = cv2.CascadeClassifier(cascalePath)

font = cv2.FONT_HERSHEY_SIMPLEX

id = 0
names = ["0","Ngoc Crazy"]

cam = cv2.VideoCapture('http://192.168.0.104:8080/video')
cam.set(3, 640)
cam.set(4, 480)

minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)

while 1:
    _, img = cam.read()
    # img = cv2.flip(img, -1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascale.detectMultiScale(
        gray,
        scaleFactor= 1.2,
        minNeighbors= 5,
        minSize= (int(minW), int(minH))
    )

    for (x, y , w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)

        id, confidence = recognizer.predict(gray[y:y+h, x:x+w])

        if confidence < 100:
            id = names[id]
            confidence = "  {0}%".format(round(100 -confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 -confidence))
        
        cv2.putText(img, str(id), (x+5, y- 5), font, 1, (255, 255, 255), 2)
        cv2.putText(img, str(confidence), (x+5, y+h-5), font, 1, (255, 255, 0), 1)

    img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
    cv2.imshow("Nhan dien khuon mat", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()